In [75]:
import os
import pandas as pd

path = "."  #absolute or relative path to the folder containing the file. 
            #"." for current folder

#read and print values from the dataset to check if it imported correctly
filename_read = os.path.join(path, "pokemon.csv")
df = pd.read_csv(filename_read)
print(df[0:5])
df.sample(frac=1).reset_index(drop=True)
print(df[0:5])

   pokedex_number        Name  Type1   Type2  hp  height_m  weight_kg  attack  \
0               1   bulbasaur  Grass  Poison  45       0.7        6.9      49   
1               2     ivysaur  Grass  Poison  60       1.0       13.0      62   
2               3    venusaur  Grass  Poison  80       2.0      100.0     100   
3               4  charmander   Fire     NaN  39       0.6        8.5      52   
4               5  charmeleon   Fire     NaN  58       1.1       19.0      64   

   defense  speed  ...  against_ghost  against_grass  against_ground  \
0       49     45  ...            1.0           0.25             1.0   
1       63     60  ...            1.0           0.25             1.0   
2      123     80  ...            1.0           0.25             1.0   
3       43     65  ...            1.0           0.50             2.0   
4       58     80  ...            1.0           0.50             2.0   

  against_ice  against_normal  against_poison  against_psychic  against_rock  \


In [43]:
#taken from tutorial 3
#df = df.select_dtypes(include=['int', 'float'])

#encode target names to integers
#https://stackoverflow.com/questions/49037286/pandas-string-values-encoding
from sklearn.preprocessing import LabelEncoder
import numpy as np

targets = pd.read_csv(filename_read)

le = LabelEncoder()
df['Type1'] = le.fit_transform(df['Type1'])

print(df['Type1'][0:20])
print(targets['Type1'][0:20])

#collect the columns names for non-target features
result = []
for x in df.columns:
    if (x == 'attack') or (x == 'defense') or (x == 'speed') or (x == 'sp_defense') or (x == 'sp_attack') or (x == 'weight'):
        result.append(x)

#get data (often called X) and target (often calle y) and display its shape
X = df[result].values
yalt = df['Type1'].values
print(X.shape)
print(yalt.shape)

from tensorflow import keras

y = keras.utils.to_categorical(df['Type1'].to_numpy())

print(y.shape)

0      9
1      9
2      9
3      6
4      6
5      6
6     17
7     17
8     17
9      0
10     0
11     0
12     0
13     0
14     0
15    12
16    12
17    12
18    12
19    12
Name: Type1, dtype: int64
0      Grass
1      Grass
2      Grass
3       Fire
4       Fire
5       Fire
6      Water
7      Water
8      Water
9        Bug
10       Bug
11       Bug
12       Bug
13       Bug
14       Bug
15    Normal
16    Normal
17    Normal
18    Normal
19    Normal
Name: Type1, dtype: object
(801, 5)
(801,)
(801, 18)


In [46]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, yalt, test_size=0.15, random_state=4)

# Create a Perceptron, with its training parameters
ppn = Perceptron(max_iter=40,tol=0.001,eta0=1)

# Train the model
ppn.fit(X_train,y_train)

# Make predication
y_pred = ppn.predict(X_test)

# Evaluate accuracy
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.11


In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

kf = KFold(6)

for train, test in kf.split(X):
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

    model.fit(X_train,y_train,verbose=0,epochs=128)
    pred = model.predict(X_test)
    pred = np.argmax(pred,axis=1)
    y_compare = np.argmax(y_test,axis=1) 
    score = metrics.accuracy_score(y_compare, pred)
    print("Accuracy score: {}".format(score))

Accuracy score: 0.26119402985074625
Accuracy score: 0.17164179104477612
Accuracy score: 0.26119402985074625
Accuracy score: 0.3233082706766917
Accuracy score: 0.17293233082706766
Accuracy score: 0.18796992481203006


In [74]:
from sklearn import metrics

pred = model.predict(X[0:20]) # using all the data
#print(pred[0:10]) # print first ten predictions

print(np.argmax(pred[0:20],axis=1))
print(np.argmax(y[0:20],axis=1))
print(targets['Type1'][0:20])

[17 17 17  6  6  6 17 17 17 12  0 14 12  9  0 12 13  0 12 13]
[ 9  9  9  6  6  6 17 17 17  0  0  0  0  0  0 12 12 12 12 12]
0      Grass
1      Grass
2      Grass
3       Fire
4       Fire
5       Fire
6      Water
7      Water
8      Water
9        Bug
10       Bug
11       Bug
12       Bug
13       Bug
14       Bug
15    Normal
16    Normal
17    Normal
18    Normal
19    Normal
Name: Type1, dtype: object
